In [1]:
import torch
#import gradio as gr

from textwrap import fill
from IPython.display import Markdown, display

from langchain.prompts.chat import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    SystemMessagePromptTemplate,
    )

from langchain import PromptTemplate
from langchain.chains import ConversationChain
from langchain import HuggingFacePipeline

from langchain.vectorstores import Chroma
from langchain.schema import AIMessage, HumanMessage
from langchain.memory import ConversationBufferMemory
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import UnstructuredMarkdownLoader, UnstructuredURLLoader
from langchain.chains import LLMChain, SimpleSequentialChain, RetrievalQA, ConversationalRetrievalChain

from transformers import BitsAndBytesConfig, AutoModelForCausalLM, AutoTokenizer, GenerationConfig, pipeline, AutoConfig

import warnings
warnings.filterwarnings('ignore')
print(torch.cuda.is_available())
num_chapters = 4

True


In [3]:
MODEL_NAME = "mistralai/Mistral-7B-Instruct-v0.1"
#MODEL_NAME = "NousResearch/Yarn-Mistral-7b-128k"

quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
)

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=True, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME, torch_dtype=torch.float16,
    #use_flash_attention = True,
    config=config,
    trust_remote_code=True,
    device_map="auto",
    quantization_config=quantization_config
)


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin C:\Users\phili\miniconda3\envs\ChildrensBook\Lib\site-packages\bitsandbytes\libbitsandbytes_cuda121_nocublaslt.dll
CUDA SETUP: CUDA runtime path found: C:\Users\phili\miniconda3\envs\ChildrensBook\bin\cudart64_12.dll
CUDA SETUP: Highest compute capability among GPUs detected: 6.1
CUDA SETUP: Detected CUDA version 121
CUDA SETUP: Loading binary C:\Users\phili\miniconda3\envs\ChildrensBook\Lib\site-packages\bitsandbytes\libbitsandbytes_cuda121_nocublaslt.dll...


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
# name = 'mosaicml/mpt-7b-storywriter'
# 
# config = AutoConfig.from_pretrained(name, trust_remote_code=True)
# # config.attn_config['attn_impl'] = 'triton'
# config.init_device = 'cuda:0' 
# config.max_seq_len = 83968 # (input + output) tokens can now be up to 83968
# quantization_config = BitsAndBytesConfig(
#     load_in_4bit=True,
#     bnb_4bit_compute_dtype=torch.float16,
#     bnb_4bit_quant_type="nf4",
#     bnb_4bit_use_double_quant=True,
# )
# 
# model = AutoModelForCausalLM.from_pretrained(
#   name,
#   config=config,
#   torch_dtype=torch.bfloat16, # Load model weights in bfloat16
#   trust_remote_code=True,
#   device_map="auto",
#   quantization_config=quantization_config
# )
# tokenizer = AutoTokenizer.from_pretrained("EleutherAI/gpt-neox-20b")


In [7]:
generation_config = GenerationConfig.from_pretrained(MODEL_NAME)
generation_config.max_new_tokens = 2000
generation_config.max_length = 10000
generation_config.top_p = 1
generation_config.do_sample = True
generation_config.temperature = 0.75
generation_config.repetition_penalty = 1

pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    return_full_text=True,
    generation_config=generation_config,
)
pipeline.model.config.pad_token_id = pipeline.model.config.eos_token_id

ValueError: The following `model_kwargs` are not used by the model: ['model', 'tokenizer'] (note: typos in the generate arguments will also show up in this list)

In [8]:
llm = HuggingFacePipeline(
    pipeline=pipeline,
    )

In [9]:
longchat_template = """<s>[INST]
Objective: Create a 4-chapter children's book set in a magical Korean world with dragons. Each 500-word chapter features a unique, open-ended adventure.
AI's Role:
Craft engaging, informative stories for young readers.
Use a friendly tone, rich descriptions, diverse characters.
Focus on positive themes like friendship and learning.
Mix in humor and educational elements.
Build resilience and empathy through challenges.
End stories with hope or lessons, avoiding negativity.
Process: Write one detailed chapter at a time, awaiting user prompts for subsequent chapters.

Current conversation:
{history}
USER: {input}
AI: [/INST]
"""

In [10]:
longchat_prompt_template = PromptTemplate(
    input_variables=["input", "history"], template=longchat_template
)

In [11]:
from langchain.chains.conversation.memory import ConversationBufferMemory

conversation_buf = ConversationChain(
    llm=llm,
    memory=ConversationBufferMemory(ai_prefix="AI", human_prefix="USER"),
    prompt=longchat_prompt_template,
)

In [12]:
# from langchain.chains.conversation.memory import ConversationSummaryMemory
# 
# conversation_buf = ConversationChain(
# 	llm=llm,
# 	memory=ConversationSummaryMemory(llm=llm, ai_prefix="AI", human_prefix="USER"),
#   prompt=longchat_prompt_template
# )

In [13]:
# from langchain.memory import ConversationSummaryBufferMemory
# 
# conversation_buf = ConversationChain(
#     llm=llm, 
#     prompt=longchat_prompt_template,
#     memory=ConversationSummaryBufferMemory(
#         llm=llm,
#         max_token_limit=1300,
#         ai_prefix="AI", 
#         human_prefix="USER")
# )

In [14]:
# from langchain.memory import ConversationKGMemory
# 
# conversation_buf = ConversationChain(
# 	llm=llm,
# 	memory=ConversationKGMemory(llm=llm),
#   prompt=longchat_prompt_template
# )

In [16]:
print(conversation_buf("Write now only the first chapter and give it a title!")["response"])

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.



Chapter 1: The Dragon Kingdom Discovery

Title: "The Secret Garden of Dragons"

Once upon a time, in a far-off land filled with lush greenery and sparkling rivers, there lived a young boy named Jae. Jae lived with his family in a small village surrounded by forests and mountains. Despite living near such breathtaking nature, Jae felt like he was missing out on the adventure of a lifetime.

One sunny morning, while exploring the forest near his home, Jae stumbled upon a hidden pathway. Curiosity piqued, he followed the winding trail deeper into the woods. As he walked, he noticed the trees becoming tall and twisted, the air growing thicker and cooler. Suddenly, the sound of rushing water caught his attention.

Following the sound, Jae came across a beautiful waterfall cascading down into a crystal-clear pool. Surrounding the pool was a lush garden overflowing with colorful flowers and exotic plants. Jae was amazed by the beauty of the garden and decided to explore further.

As he walke

In [ ]:
print(conversation_buf(f"[INST] Give me 4 decision possibilities of how the story could continue! Write no more than two sentences. [/INST]")["response"])

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


In [ ]:
import random
num_dec = 4  # number of decisions
letters = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j']
if num_dec > len(letters):
    num_dec = len(letters)
for i in range(num_chapters-2):
    decision = random.choice(letters[:num_dec])
    print(f"Decision: {decision}")
    print(conversation_buf(f" Continue the story with decision {decision}. Create only the next chapter now! Keep in mind to write at least 500 words! [/INST]")["response"])
    print("%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%")
    print(conversation_buf(f"[INST] Give me four decision possibilities of how the story could continue. Write no more than two sentences. [/INST]")["response"])

In [ ]:
decision = "c"
print(conversation_buf(f"[INST] Continue the story with decision {decision}. Create only the next and last chapter now! Keep in mind to write at least 500 words! Find a happy or sad ending based on the previous decisions. [/INST]")["response"])

In [ ]:
print(conversation_buf.memory.buffer)